In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-25 18:47:44.505355: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 18:47:44.529899: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 18:47:44.529924: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 18:47:44.529949: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 18:47:44.535256: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-25 18:47:46.156920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:47:46.180591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:47:46.183472: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:

"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [4]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [5]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [6]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [7]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-25 18:47:46.659001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:47:46.661678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 18:47:46.664430: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [8]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [9]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

In [10]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.EfficientNetB7(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: rescaling, False
Layer: normalization, False
Layer: rescaling_1, False
Layer: stem_conv_pad, False
Layer: stem_conv, False
Layer: stem_bn, False
Layer: stem_activation, False
Layer: block1a_dwconv, False
Layer: block1a_bn, False
Layer: block1a_activation, False
Layer: block1a_se_squeeze, False
Layer: block1a_se_reshape, False
Layer: block1a_se_reduce, False
Layer: block1a_se_expand, False
Layer: block1a_se_excite, False
Layer: block1a_project_conv, False
Layer: block1a_project_bn, False
Layer: block1b_dwconv, False
Layer: block1b_bn, False
Layer: block1b_activation, False
Layer: block1b_se_squeeze, False
Layer: block1b_se_reshape, False
Layer: block1b_se_reduce, False
Layer: block1b_se_expand, False
Layer: block1b_se_excite, False
Layer: block1b_project_conv, False
Layer: block1b_project_bn, False
Layer: block1b_drop, False
Layer: block1b_add, False
Layer: block1c_dwconv, False
Layer: block1c_bn, False
Layer: block1c_activation, False
Layer: block1c_se_sque

In [13]:
len(base_model.layers)

814

In [17]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [18]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [19]:
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = aug_layer(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'EfficientB7_90.49')

model_1.summary()

Model: "EfficientB7_90.49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_1  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 256)               655616    
                                                 

In [24]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models','EfficientB7'), save_weights_only = False, save_best_only = True, verbose = 1)

In [25]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=2) 

In [26]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [ ]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10
74/74 [==============================] - ETA: 0s - loss: 0.8890 - accuracy: 0.6205
Epoch 1: val_loss improved from inf to 1.16254, saving model to Trained_Models/EfficientB7
74/74 [==============================] - 120s 2s/step - loss: 0.8890 - accuracy: 0.6205 - val_loss: 1.1625 - val_accuracy: 0.4125 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - ETA: 0s - loss: 0.7583 - accuracy: 0.6895
Epoch 2: val_loss did not improve from 1.16254
74/74 [==============================] - 41s 548ms/step - loss: 0.7583 - accuracy: 0.6895 - val_loss: 1.3840 - val_accuracy: 0.2997 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - ETA: 0s - loss: 0.6964 - accuracy: 0.7200
Epoch 3: val_loss improved from 1.16254 to 1.14368, saving model to Trained_Models/EfficientB7


In [20]:
base_model.trainable = True

In [21]:
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [22]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [23]:
start_epoch = 10
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 2560)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               655616    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               

In [24]:
history_25 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/20
74/74 [==============================] - ETA: 0s - loss: 1.4415 - accuracy: 0.5426
Epoch 11: val_loss did not improve from 0.28987
74/74 [==============================] - 27s 224ms/step - loss: 1.4415 - accuracy: 0.5426 - val_loss: 0.5542 - val_accuracy: 0.8309 - lr: 0.0010
Epoch 12/20
74/74 [==============================] - ETA: 0s - loss: 0.5825 - accuracy: 0.7874
Epoch 12: val_loss did not improve from 0.28987
74/74 [==============================] - 14s 191ms/step - loss: 0.5825 - accuracy: 0.7874 - val_loss: 1.0118 - val_accuracy: 0.5865 - lr: 0.0010
Epoch 13/20
74/74 [==============================] - ETA: 0s - loss: 0.4270 - accuracy: 0.8484
Epoch 13: val_loss did not improve from 0.28987
74/74 [==============================] - 15s 195ms/step - loss: 0.4270 - accuracy: 0.8484 - val_loss: 0.6098 - val_accuracy: 0.8556 - lr: 0.0010
Epoch 14/20
74/74 [==============================] - ETA: 0s - loss: 0.3018 - accuracy: 0.8899
Epoch 14: val_loss did not improve from 0

In [25]:
start_epoch = 20
for layer in base_model.layers[-175:]:
    layer.trainable = True

In [26]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0005), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 2560)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               655616    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               

In [27]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
74/74 [==============================] - ETA: 0s - loss: 1.6789 - accuracy: 0.6095
Epoch 21: val_loss did not improve from 0.13556
74/74 [==============================] - 38s 249ms/step - loss: 1.6789 - accuracy: 0.6095 - val_loss: 0.4152 - val_accuracy: 0.8912 - lr: 7.5000e-04
Epoch 22/30
74/74 [==============================] - ETA: 0s - loss: 0.5349 - accuracy: 0.7789
Epoch 22: val_loss did not improve from 0.13556
74/74 [==============================] - 17s 226ms/step - loss: 0.5349 - accuracy: 0.7789 - val_loss: 0.5752 - val_accuracy: 0.8111 - lr: 7.5000e-04
Epoch 23/30
74/74 [==============================] - ETA: 0s - loss: 0.7716 - accuracy: 0.7975
Epoch 23: val_loss did not improve from 0.13556
74/74 [==============================] - 17s 227ms/step - loss: 0.7716 - accuracy: 0.7975 - val_loss: 0.4448 - val_accuracy: 0.8556 - lr: 7.5000e-04
Epoch 24/30
74/74 [==============================] - ETA: 0s - loss: 0.3127 - accuracy: 0.8916
Epoch 24: val_loss did not im

In [28]:
start_epoch = 30
for layer in base_model.layers[-350:]:
    layer.trainable = True

In [29]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000075), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 2560)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               655616    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               

In [ ]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/40


In [33]:
start_epoch = 35
for layer in base_model.layers[-400:]:
    layer.trainable = True

In [35]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00025), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 2560)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               327808    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 11)                

In [ ]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 26/35
380/380 [==============================] - ETA: 0s - loss: 0.2814 - accuracy: 0.8822
Epoch 26: val_loss did not improve from 0.15705
380/380 [==============================] - 102s 230ms/step - loss: 0.2814 - accuracy: 0.8822 - val_loss: 0.1877 - val_accuracy: 0.9431 - lr: 2.5000e-04
Epoch 27/35
379/380 [============================>.] - ETA: 0s - loss: 0.2215 - accuracy: 0.9062
Epoch 27: val_loss did not improve from 0.15705
380/380 [==============================] - 83s 219ms/step - loss: 0.2223 - accuracy: 0.9059 - val_loss: 0.2635 - val_accuracy: 0.9300 - lr: 2.5000e-04
Epoch 28/35
380/380 [==============================] - ETA: 0s - loss: 0.1747 - accuracy: 0.9258
Epoch 28: val_loss improved from 0.15705 to 0.14297, saving model to Trained_Models/EfficientB7
380/380 [==============================] - 125s 330ms/step - loss: 0.1747 - accuracy: 0.9258 - val_loss: 0.1430 - val_accuracy: 0.9544 - lr: 2.5000e-04
Epoch 29/35
380/380 [==============================] - ETA: 0s

In [11]:
effb7_97 = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/EfficientB7_90.49")


In [12]:
effb7_97.evaluate(test_datagen)

2024-07-25 18:48:13.389220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-25 18:48:13.444363: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


27/27 [==============================] - 7s 138ms/step - loss: 0.3847 - accuracy: 0.9041


[0.3847275972366333, 0.9041420221328735]